In [1]:
Text="I am learning NLP"

In [3]:
import pandas as pd
pd.get_dummies(Text.split())

,I,NLP,am,learning
0,True,False,False,False
1,False,False,True,False
2,False,False,False,True
3,False,True,False,False


In [5]:
text = ["i love NLP and i will learn in 2month"]

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(text)
vector = vectorizer.transform(text)

In [11]:
print(vectorizer.vocabulary_)
print(vector.toarray())

{'love': 4, 'nlp': 5, 'and': 1, 'will': 6, 'learn': 3, 'in': 2, '2month': 0}
[[1 1 1 1 1 1 1]]


In [13]:
print(vector)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 7 stored elements and shape (1, 7)>
  Coords	Values
  (0, 0)	1
  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1


In [15]:
CountVectorizer?

Init signature:
CountVectorizer(
    *,
    input='content',
    encoding='utf-8',
    decode_error='strict',
    strip_accents=None,
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    stop_words=None,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 1),
    analyzer='word',
    max_df=1.0,
    min_df=1,
    max_features=None,
    vocabulary=None,
    binary=False,
    dtype=<class 'numpy.int64'>,
)
Docstring:     
Convert a collection of text documents to a matrix of token counts.

This implementation produces a sparse representation of the counts using
scipy.sparse.csr_matrix.

If you do not provide an a-priori dictionary and you do not use an analyzer
that does some kind of feature selection then the number of features will
be equal to the vocabulary size found by analyzing the data.

For an efficiency comparison of the different feature extractors, see
:ref:`sphx_glr_auto_examples_text_plot_hashing_vs_dict_vectorizer.py`.

Read more in the :ref:`User Guide

In [17]:
df = pd.DataFrame(data=vector.toarray(), columns=vectorizer.get_feature_names_out())

In [19]:
df

,2month,and,in,learn,love,nlp,will
0,1,1,1,1,1,1,1


In [33]:
text="I am learning NLP"

In [43]:
from textblob import TextBlob
TextBlob(text).ngrams(2)

[WordList(['I', 'am']),
 WordList(['am', 'learning']),
 WordList(['learning', 'NLP'])]

In [25]:
!pip install textblob

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1


In [39]:
!python -m textblob.download_corpora

Finished.


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [45]:
TextBlob(text).ngrams(3)

[WordList(['I', 'am', 'learning']), WordList(['am', 'learning', 'NLP'])]

In [47]:
TextBlob(text).ngrams(4)

[WordList(['I', 'am', 'learning', 'NLP'])]

# Covid Fake New Classification

In [2]:
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle
from sklearn.linear_model import LogisticRegressionCV
import re
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
df=pd.read_csv('covid_fake.csv')

In [6]:
df.head()

,title,text,source,label
0,Due to the recent outbreak for the Coronavirus...,"You just need to add water, and the drugs and ...",coronavirusmedicalkit.com,Fake
1,NaN,Hydroxychloroquine has been shown to have a 10...,RudyGiuliani,Fake
2,NaN,Fact: Hydroxychloroquine has been shown to hav...,CharlieKirk,Fake
3,NaN,The Corona virus is a man made virus created i...,JoanneWrightForCongress,Fake
4,NaN,Doesn’t @BillGates finance research at the Wuh...,JoanneWrightForCongress,Fake


In [8]:
df.shape

(1164, 4)

In [10]:
df['label'].value_counts()

label
TRUE    584
Fake    345
fake    230
Name: count, dtype: int64

In [12]:
df.loc[5:15]

,title,text,source,label
5,CORONA UNMASKED: Chinese Intelligence Officer ...,NaN,NaN,NaN
6,NaN,Urgent: Health Bulletin to the Public. Ministr...,Ministry of Health,Fake
7,NaN,"Pls tell ur families, relatives and friendsMOH...",NWLLAB,Fake
8,NaN,SERIOUS EXCELLENT ADVICE by Japanese doctors t...,Japanese doctors treating COVID-19 cases,Fake
9,Basic protective measures against the new coro...,Stay aware of the latest information on the CO...,https://www.who.int/emergencies/diseases/novel...,TRUE
10,NaN,The new Coronavirus may not show signs of infe...,Taiwan Experts,Fake
11,NaN,A vaccine meant for cattle can be used to figh...,facebook,Fake
12,NaN,Using a hair dryer to breathe in hot air can c...,Youtube,Fake
13,NaN,Corona virus before it reaches the lungs it re...,twitter,Fake
14,Exposing yourself to the sun or to temperature...,"You can catch COVID-19, no matter how sunny or...",https://www.who.int/emergencies/diseases/novel...,TRUE


In [14]:
# checking andfilling the missing values

df.loc[df['label'] == 'Fake', ['label']] = 'FAKE' 
df.loc[df['label'] == 'fake', ['label']] = 'FAKE' 
df.loc[df['source'] == 'facebook', ['source']] = 'Facebook' 
df.text.fillna(df.title, inplace=True) 
df.loc[5]['label'] = 'FAKE' 
df.loc[15]['label'] = 'TRUE' 
df.loc[43]['label'] = 'FAKE' 
df.loc[131]['label'] = 'TRUE' 
df.loc[242]['label'] = 'FAKE'  
df.title.fillna('missing', inplace=True) 
df.source.fillna('missing', inplace=True) 
df['title_text'] = df['title'] + ' ' + df['text']

In [16]:
df.isna().sum()

title         0
text          0
source        0
label         0
title_text    0
dtype: int64

In [20]:
df['label'].value_counts()

label
TRUE    586
FAKE    578
Name: count, dtype: int64

In [22]:
df.head()

,title,text,source,label,title_text
0,Due to the recent outbreak for the Coronavirus...,"You just need to add water, and the drugs and ...",coronavirusmedicalkit.com,FAKE,Due to the recent outbreak for the Coronavirus...
1,missing,Hydroxychloroquine has been shown to have a 10...,RudyGiuliani,FAKE,missing Hydroxychloroquine has been shown to h...
2,missing,Fact: Hydroxychloroquine has been shown to hav...,CharlieKirk,FAKE,missing Fact: Hydroxychloroquine has been show...
3,missing,The Corona virus is a man made virus created i...,JoanneWrightForCongress,FAKE,missing The Corona virus is a man made virus c...
4,missing,Doesn’t @BillGates finance research at the Wuh...,JoanneWrightForCongress,FAKE,missing Doesn’t @BillGates finance research at...


In [26]:
df.shape

(1164, 5)

In [28]:
df['title_text'][3]

'missing The Corona virus is a man made virus created in a Wuhan laboratory. Ask @BillGates who financed it.'

In [32]:
import re 
import pandas as pd
def preprocessor(text):
  text = re.sub('<[^>]*>', '',text)
  text = re.sub(r'[^\w\s]', '',text)
  text = re.sub(r'[\n]', '',text)
  text = text.lower()
  return text
df['title_text'] = df['title_text'].apply(preprocessor)
df['title_text'][3]

'missing the corona virus is a man made virus created in a wuhan laboratory ask billgates who financed it'

In [34]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
  return [porter.stem(word) for word in text.split()]

In [44]:
tfidf = TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None,tokenizer=tokenizer_porter,use_idf=True,norm='l2',smooth_idf=True)
X=tfidf.fit_transform(df['title_text'])
y=df.label.values
X.shape


(1164, 27020)

In [42]:
X.shape

(1164, 27020)

In [56]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0, test_size=0.3,shuffle=False)

In [58]:
from sklearn.linear_model import LogisticRegressionCV
clf =  LogisticRegressionCV(cv=5, scoring='accuracy', random_state=0, n_jobs=-1, verbose=0, max_iter=300)
clf.fit(X_train, y_train)
fake_news_model = open('fake_news_model.sav', 'wb')
pickle.dump(clf, fake_news_model)
fake_news_model.close()

In [59]:
filename= 'fake_news_model.sav'
saved_clf = pickle.load(open(filename, 'rb'))
saved_clf.score(X_test, y_test)

0.9314285714285714

In [62]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = saved_clf.predict(X_test)
print("_Test Set Results_")
print("Accuracy: {}".format(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))

_Test Set Results_
Accuracy: 0.9314285714285714
              precision    recall  f1-score   support

        FAKE       0.92      0.89      0.91       132
        TRUE       0.94      0.95      0.95       218

    accuracy                           0.93       350
   macro avg       0.93      0.92      0.93       350
weighted avg       0.93      0.93      0.93       350



In [66]:
clf.predict(X_test[5])

array(['TRUE'], dtype=object)

In [68]:
test = "Corona virus before it reaches the lungs"
inp = [test]
vect = tfidf.transform(inp)
predicition = clf.predict(vect)
print(predicition)

['FAKE']
